In [1]:
# !wget https://huggingface.co/datasets/mesolitica/malaysian-ultrachat/resolve/main/ultrachat-lom-agc.jsonl

In [2]:
from transformers import AutoModel, AutoTokenizer
import torch
import json
from tqdm import tqdm
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [3]:
model = AutoModel.from_pretrained('mesolitica/llama2-embedding-1b-8k', 
                                  trust_remote_code = True,
                                 use_flash_attention_2 = True, torch_dtype = torch.bfloat16)

[2023-11-26 05:43:57,029] [INFO] [real_accelerator.py:133:get_accelerator] Setting ds_accelerator to cuda (auto detect)


You are attempting to use Flash Attention 2.0 with a model initialized on CPU. Make sure to move the model to GPU after initializing it on CPU with `model.to('cuda')`.


In [4]:
_ = model.cuda()

In [5]:
tokenizer = AutoTokenizer.from_pretrained('mesolitica/llama2-embedding-1b-8k')

In [6]:
contexts, questions = [], []

with open('ultrachat-lom-agc.jsonl') as fopen:
    for l in fopen:
        l = json.loads(l)
        if l[1]['content_ms']:
            contexts.append(l[0]['content'].strip())
            questions.append(l[1]['content_ms'].strip())
            
len(contexts), len(questions)

(8034, 8034)

In [7]:
contexts_v = []
for i in tqdm(range(len(contexts))):
    input_ids = tokenizer([contexts[i]], 
        return_tensors = 'pt',
        padding = True
    )
    v = model.encode(input_ids.to('cuda')).detach().cpu().numpy()
    contexts_v.append(v)

100%|██████████| 8034/8034 [02:17<00:00, 58.40it/s]


In [8]:
questions_v = []
for i in tqdm(range(len(questions))):
    input_ids = tokenizer([questions[i]], 
        return_tensors = 'pt',
        padding = True
    )
    v = model.encode(input_ids.to('cuda')).detach().cpu().numpy()
    questions_v.append(v)

100%|██████████| 8034/8034 [00:45<00:00, 175.87it/s]


In [9]:
contexts_v_np = np.array(contexts_v)[:,0]
questions_v_np = np.array(questions_v)[:,0]

In [10]:
tops = {
    1: 0,
    3: 0,
    5: 0,
    10: 0,
}

for i in tqdm(range(len(questions_v_np))):
    argsort = np.argsort(cosine_similarity(questions_v_np[i].reshape(1, -1), contexts_v_np)[0])[::-1]
    for k in tops.keys():
        if i in argsort[:k]:
            tops[k] += 1

100%|██████████| 8034/8034 [03:40<00:00, 36.39it/s]


In [11]:
for k, v in tops.items():
    print(k, v / len(questions_v_np))

1 0.06870799103808813
3 0.1343042071197411
5 0.1717699775952203
10 0.23089370176748816
